In [4]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [5]:
from chromalab.spectra import Spectra, Illuminant, convert_refs_to_spectras
from chromalab.observer import Observer
from chromalab.maxbasis import MaxBasis
from chromalab.spectral_analysis import NTIREDataLoader, RudermanAnalysis
from chromalab.visualizer import PSWrapper, DisplayBasisType, exportAndPlay


objc[91675]: Class GLFWApplicationDelegate is implemented in both /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/open3d/cpu/pybind.cpython-311-darwin.so (0x29e7c2e28) and /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/polyscope_bindings.cpython-311-darwin.so (0x2b9ec2ea0). One of the two will be used. Which one is undefined.
objc[91675]: Class GLFWWindowDelegate is implemented in both /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/open3d/cpu/pybind.cpython-311-darwin.so (0x29e7c2db0) and /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/polyscope_bindings.cpython-311-darwin.so (0x2b9ec2ec8). One of the two will be used. Which one is undefined.
objc[91675]: Class GLFWContentView is implemented in both /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/open3d/cpu/pybind.cpython-311-darwin.so (0x29e7c2e50) and /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-

In [6]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

In [23]:
wavelengths10 = np.arange(400, 701, 10)
d65 = Illuminant.get("D65")

trichromat = Observer.trichromat(wavelengths=wavelengths10, illuminant=d65) 
maxbasis_tri = MaxBasis(trichromat, verbose=True)
LMS_to_RGB = maxbasis_tri.get_cone_to_maxbasis_transform()

tetrachromat = Observer.tetrachromat(wavelengths=wavelengths10, illuminant=d65, verbose=True)
maxbasis = MaxBasis(tetrachromat, verbose=True)
viz = PSWrapper(tetrachromat, maxbasis, itemsToDisplay=PSWrapper.ItemsToDisplay.MESH, displayBasis=DisplayBasisType.CONE, verbose=True)

100%|██████████| 28/28 [00:00<00:00, 942.00it/s]
/Users/jessicalee/Projects/generalized-colorimetry/code/ChromaLab/chromalab/spectra.py:36: UserWarning: Data has values not between 0 and 1. Clipping.
  warnings.warn("Data has values not between 0 and 1. Clipping.")
100%|██████████| 28/28 [00:00<00:00, 63.65it/s]


In [8]:
# Process NTIRE Data
# ntire_data_loader = NTIREDataLoader("/data/NTIRE_2022", tetrachromat)
# ntire_data_loader.getConeResponses()
# ntire_data_loader.getSpatialConeResponses()
# ntire_data_loader.save("./data/ntire_data_loader.pkl")

# Load NTIRE Data
ntire_data_loader = NTIREDataLoader.load("./data/ntire_data_loader.pkl")

In [9]:
tetrachromat_analysis = RudermanAnalysis(tetrachromat, ntire_data_loader)

In [24]:
tetrachromat_analysis.doConeResponsePCA()
tetrachromat_analysis.plotPCA4D(viz, LMS_to_RGB)
viz.ps.show()

[[ 0.49793299  0.50450443  0.50035148  0.49717845]
 [ 0.86178678 -0.20338577 -0.29933659 -0.35546501]
 [-0.09675478  0.76733862 -0.04946008 -0.63196808]
 [ 0.00497864 -0.33955994  0.81092521 -0.47652341]]
[  9.62180134e-01   3.59249986e-02   1.89164777e-03   3.21939766e-06]


In [ ]:
tetrachromat_analysis.doSpatialPCA()
# displays the Spatial PCA results first plotting in SML, and then in MQL space (the long wavelength cone space)
tetrachromat_analysis.displaySpatialPCA("./outputs/spatial_pca.png")